#### Binance data scrapping using BTools
##### Author: Mickael de Oliveira
###### Example of BTools for easily acess to exchanges APIs
###### See more in: TODO

#### Importing libs

In [1]:
from BTools import *
import pandas as pd

#### Constants section

In [2]:
TIMESTART = 1514689200000 # 2018-01-01 00:00:00
TIMEEND = 1672542000000 # 2023-01-01 00:00:00

INTERVAL = "15m"
DEFAULTPAIR = "BTCUSDT"

DELTA = TIMEEND - TIMESTART

CANDLES = DELTA//getTime(INTERVAL)

#### Creating base dataframe with bitcoin price 

In [3]:
dados = []
for i in getCandlestick(TIMESTART,DEFAULTPAIR,INTERVAL,CANDLES,0):
    dados.append({"timestamp": i[0], "volumeRisky" : 0, "volumeSafe": i[3],"tradesRisky": 0, "tradesSafe": i[4], "price": i[2]})
    
data = pd.DataFrame(dados)
data

,timestamp,volumeRisky,volumeSafe,tradesRisky,tradesSafe,price,flag
0,1514689200000,0,201.850986,0,2356,13162.47,0
1,1514690100000,0,116.966077,0,1437,13399.95,0
2,1514691000000,0,136.624371,0,1644,13225.85,0
3,1514691900000,0,181.561263,0,1858,12962.06,0
4,1514692800000,0,124.738029,0,1203,12896.00,0
...,...,...,...,...,...,...,...
175387,1673011800000,0,8656.382630,0,173424,16777.93,0
175388,1673012700000,0,4540.690760,0,106208,16774.19,0
175389,1673013600000,0,2330.001460,0,67331,16756.22,0
175390,1673014500000,0,2250.175950,0,62809,16762.26,0


#### Searching for remanescent pairs from 2018 and appending to database

In [4]:
mPair = pairs()
for i in mPair.get("BTC",timeStart=TIMESTART, timeEnd=TIMEEND):
    if str(i) != "BTCUSDT":
        if i.getBase() == "BTC":
            control = 0
        else:
            control = 1
            
        if i.isSafe():
            count = 0
            for j in getCandlestick(TIMESTART,i,INTERVAL,CANDLES,control):
                data.loc[count,"volumeSafe"] += j[3]
                data.loc[count,"tradesSafe"] += j[4]
                count += 1
        
        else:
            count = 0
            for j in getCandlestick(TIMESTART,i,INTERVAL,CANDLES,control):
                data.loc[count,"volumeRisky"] += j[3]
                data.loc[count,"tradesRisky"] += j[4]
                count += 1

data

#### Finding volume in Dollars to make more intuitive data

In [ ]:
for index, row in data.iterrows():
    data.loc[index,"volumeRisky"] *= data.loc[index,"price"]
    data.loc[index,"volumeSafe"] *= data.loc[index,"price"]
data

In [ ]:
data.to_csv("data.csv",index=False)